In [2]:
import os
import numpy as np
import gzip
import pickle

import torch
from torch_geometric.data import Dataset, Data, DataLoader

import import_ipynb
from influenceDataset import get_dataloader_10000_noprocessing
from gnnNets import get_gnnNets

importing Jupyter notebook from influenceDataset.ipynb
importing Jupyter notebook from gnnNets.ipynb


In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "3" 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [8]:
GPU_NUM = 0

device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)

print(device)

cuda:0


In [11]:
lr = 1e-3
epochs = 1000
batch_size = 20

In [5]:
# 나중에 checkpoint불러오는것과 버전별로 저장 구현하기

train_dataloader, val_dataloader, test_dataloader = get_dataloader_10000_noprocessing(batch_size)

model = get_gnnNets(1, 1, {'gnn_latent_dim':[128,128,128]})
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss(reduction='sum')

best_val_loss = float('inf')
best_model_state_dict = None

/opt/conda/envs/URP/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


8000 1000 1000


In [12]:
# torch.save 주석처리해둠.

for epoch in range(epochs):
    if epoch%10==0 : print(f'----- epoch {epoch+1} / {epochs} -----')
    # train
    model.train()
    train_loss = 0
    for batch in train_dataloader:
        batch = batch.to(device)
        
        preds = model(batch)
        
        loss = loss_fn(preds,batch.y)
        train_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
    train_loss /= len(train_dataloader.dataset)
    
    # validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for batch in val_dataloader:
            batch = batch.to(device)
            
            preds = model(batch)
            
            loss = loss_fn(preds,batch.y)
            val_loss += loss.item()
        val_loss /= len(val_dataloader.dataset)
    
    if best_val_loss > val_loss:
        best_val_loss = val_loss
        best_model_state_dict = model.state_dict()
    
    if epoch%10==0 :
        print('train loss:',train_loss**0.5)
        print('val loss:',val_loss**0.5)
print('training end\n')
    
model.load_state_dict(best_model_state_dict)
torch.save(best_model_state_dict, "/data/URP/model/"+'bestmodel.pt')

        
# test
model.eval()
with torch.no_grad():
    test_loss = 0
    for batch in test_dataloader:
        batch = batch.to(device)
        
        preds = model(batch)
        
        loss = loss_fn(preds,batch.y)
        test_loss += loss.item()
    test_loss /= len(test_dataloader.dataset)
print('test loss:', test_loss**0.5)

----- epoch 1 / 1000 -----
train loss: 453.33534700456573
val loss: 447.83849320932654
----- epoch 11 / 1000 -----
train loss: 453.29131968304887
val loss: 447.78238911328344
----- epoch 21 / 1000 -----
train loss: 453.28967194002996
val loss: 447.7927785259606
----- epoch 31 / 1000 -----
train loss: 453.30610186095555
val loss: 447.9173280305195
----- epoch 41 / 1000 -----
train loss: 453.2246348045856
val loss: 447.9233117956689
----- epoch 51 / 1000 -----
train loss: 453.28874674083295
val loss: 448.0010172979521


KeyboardInterrupt: 

In [ ]:
# 100 epoch에 30분정도
# 한 250epoch쯤 되면 loss 450정도에서 멈춤

In [13]:
model.eval()
with torch.no_grad():
    test_loss = 0
    for batch in test_dataloader:
        batch = batch.to(device)
        
        preds = model(batch)
        
        loss = loss_fn(preds,batch.y)
        print(preds,batch.y)
        test_loss += loss.item()
    test_loss /= len(test_dataloader.dataset)
print('test loss:', test_loss**0.5)

tensor([[793.2678],
        [798.6112],
        [793.2678],
        [793.2678],
        [793.2678],
        [793.2678],
        [793.2678],
        [793.2678],
        [793.2678],
        [793.2678],
        [793.2678],
        [793.2678],
        [793.2678],
        [793.2678],
        [793.2678],
        [795.0762],
        [793.2678],
        [831.7569],
        [793.2678],
        [793.2678]], device='cuda:0') tensor([[ 105.1601],
        [1489.1471],
        [1539.3541],
        [ 571.4389],
        [ 863.3654],
        [1388.8416],
        [ 956.1293],
        [ 868.2684],
        [1178.4026],
        [ 269.9209],
        [1353.4868],
        [ 885.2338],
        [ 828.2694],
        [1034.8777],
        [1331.7021],
        [1189.0812],
        [1318.4443],
        [1469.0405],
        [1295.8320],
        [1333.2008]], device='cuda:0')
tensor([[793.2678],
        [793.2678],
        [793.2678],
        [793.2678],
        [793.2678],
        [793.2678],
        [804.7341],
    